In [2]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

In [10]:
model = pyo.ConcreteModel()

######Sets
model.i = pyo.Set(initialize= ["P1","P2"]) #Factory
model.j = pyo.Set(initialize =["W1","W2","W3"]) #Warehouse
model.k = pyo.Set(initialize = ["C1","C2","C3","C4"]) #customer
model.t = pyo.RangeSet(1,3) #Month

######Decision Varibles
model.F = pyo.Var(model.i , model.j, model.t , within = pyo.NonNegativeIntegers) # Factory to Warehouse
F = model.F
model.W = pyo.Var( model.j , model.k , model.t , within = pyo.NonNegativeIntegers) # Warehouse to Customers
W = model.W
model.I = pyo.Var( model.j, model.t , within = pyo.NonNegativeIntegers) # Inventory
I = model.I
model.S = pyo.Var(  model.k, model.t,  within = pyo.NonNegativeIntegers) # Shortage
S = model.S

######Parameters

##Demand

demand = {("C1",1):100 , ("C1",2):120 , ("C1",3):110 
          ,("C2",1):80 , ("C2",2):90 , ("C2",3):85 
          ,("C3",1):90 , ("C3",2):95 , ("C3",3):100
          ,("C4",1):70 , ("C4",2):80 , ("C4",3):75}

model.D = pyo.Param(model.k , model.t , initialize = demand)
D = model.D

##Costs of Transportaion

#Cost of Factory to Warehouse

FW = {("P1","W1"):5 , ("P1","W2"):6 , ("P1","W3"):4
      ,("P2","W1"):6 , ("P2","W2"):3 , ("P2","W3"):7 } 

#Cost of Warehouse to Customers

WC = {("W1","C1"):3 , ("W1","C2"):4 , ("W1","C3"):6 , ("W1","C4"):5 ,
      ("W2","C1"):4 , ("W2","C2"):3 , ("W2","C3"):5 , ("W2","C4"):6 ,
      ("W3","C1"):5 , ("W3","C2"):6 , ("W3","C3"):4 , ("W3","C4"):3 }

model.FW = pyo.Param(model.i , model.j , initialize = FW)
FW = model.FW

model.WC = pyo.Param(model.j , model.k , initialize = WC)
WC = model.WC
##Holding cost

model.HC = pyo.Param(initialize = 1)
HC = model.HC
##Penalty Cost

model.PC = pyo.Param(initialize  = 10)
PC = model.PC
##Capacity of Factories

CF = {"P1":250 , "P2":200}

model.CF = pyo.Param(model.i , initialize = CF)
CF = model.CF

##Capacity of Warehouse

CW = {"W1":300 , "W2":300 , "W3":300}

model.CW = pyo.Param(model.j , initialize = CW)
CW = model.CW


######Objective Function
def Objective_rule(m):
    transport_cost = sum(FW[i,j]*F[i,j,t] for i in model.i  for j in model.j  for t in model.t)
    transport_cost += sum(WC[j,k]*W[j,k,t] for j in model.j  for k in model.k for t in model.t)
    holding_cost = sum(I[j,t]*HC for j in model.j for t in model.t)
    plenty_cost = sum(S[k,t]*PC for k in model.k for t in model.t)
    return transport_cost + holding_cost + plenty_cost

model.Obj = pyo.Objective(rule = Objective_rule , sense = pyo.minimize)


######Constraints

##Warehouse Capacity

def Constraint1(m,j,t):
    return I[j,t] <= CW[j]
model.const1 = pyo.Constraint(model.j , model.t , rule = Constraint1)

##Customer demand

def Constraint2(m,k,t):
    return sum(W[j,k,t]for j in model.j) + S[k,t] == D[k,t]

model.const2 = pyo.Constraint(model.k,model.t , rule = Constraint2)

##Production Capacity

def Constraint3(m,i,t):
    return (sum(F[i,j,t] for j in model.j) <= CF[i])

model.const3 = pyo.Constraint(model.i , model.t , rule = Constraint3)

##Inventory Balance

def Constraint4(m,j,t):
    
    if t == 1:
        return I[j,t] == sum(F[i,j,t] for i in model.i) - sum(W[j,k,t] for k in model.k)
    else:
        return I[j,t] == I[j,t-1] + sum(F[i,j,t] for i in model.i) - sum(W[j,k,t] for k in model.k)


model.const4 = pyo.Constraint(model.j , model.t , rule = Constraint4)



######Solver

Solver = SolverFactory("glpk")
results = Solver.solve(model)

print(results)
print("OBjective Function is ",model.Obj())


for t in model.t:
    for i in model.i:
        for j in model.j:
            print("number of Tail that transport from Factory" , i,"to Warehouse",j,"is",F[i,j,t]())
            
            
            
for t in model.t:
    for j in model.j:
        for k in model.k:
            print("number of Tail that transport from Warehouse" , j,"to Customer",k,"is",W[j,k,t]())


Problem: 
- Name: unknown
  Lower bound: 7705.0
  Upper bound: 7705.0
  Number of objectives: 1
  Number of constraints: 36
  Number of variables: 75
  Number of nonzeros: 144
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.11472487449645996
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

OBjective Function is  7705.0
number of Tail that transport from Factory P1 to Warehouse W1 is 0.0
number of Tail that transport from Factory P1 to Warehouse W2 is 0.0
number of Tail that transport from Factory P1 to Warehouse W3 is 140.0
number of Tail that transport from Factory P2 to Warehouse W1 is 0.0
number of Tail that transport from Factory P2 to Warehouse W2 is 200.0
number of Tail that transport from Factory P2 to Warehouse W3 is 0.0
number of Tail that transport from Factory P1 to Warehouse W1 is 10.0
nu